In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from dataImporter import *
from datamodel import TrackSession
import folium
import math



In [2]:
sourceFile = input("Data file: ")
dataReader = getFileImporter(sourceFile)
session = dataReader.readSessionData()
session.addSessionInfo(sourcefile = sourceFile)
session.trimEnds()

Data file: sample-data/aim-datafile-1.csv


In [3]:
print("Lap times")
print("---------")
for count,lap in enumerate(session.getLapTimes()):
    print(f"Lap { count }: {math.trunc(lap/60):02}:{lap%60:0>6.3f}")


Lap times
---------
Lap 0: 03:16.050
Lap 1: 02:57.250
Lap 2: 02:53.250
Lap 3: 02:49.300
Lap 4: 02:47.750
Lap 5: 02:49.550
Lap 6: 02:45.150
Lap 7: 02:58.750


In [4]:
print('These datapoints are available:')
for point in session.getDataPointsAvail():
    print (f"- {point}")

These datapoints are available:
- time
- GPSlat
- GPSlng
- throttle
- lap
- segment


In [5]:

# Generate full session trace and plot on map
boundingBox = session.getImageBoundaries()
location = session.getMapLocation()
mapPoints = []
m = folium.Map(location=session.getMapLocation(), zoom_start=15, tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}", attr="Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community")
m.fit_bounds(boundingBox)
for lap in session.getLaps():
    mapPoints = []
    for measurement in lap:
        mapPoints.append([measurement["GPSlat"], measurement["GPSlng"]])
    folium.PolyLine(mapPoints).add_to(m)
m


In [6]:
# Generate individual segment traces, plot on different maps.
maps = []
for segment in range(1, len(session.waypoints)+1):
    segmentNum = segment
    traces = sorted(session.getSegmentsByTime(segmentNum), key=lambda x: x['time'])

    # if this is the last segment and the fastest lap is the last lap, discard it because that's the in-segment
    # and we want to see the actual fasted during-segment. The in-segment is likely to be the fastest simply because
    # it's so much shorter, even if we are slower
    if segmentNum == len(session.waypoints) and session.numLaps == traces[0]["lap"]:
        del traces[0]

    m = folium.Map(location=session.getSeriesCenterpoint(traces[0]["path"]), zoom_start=15, tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}", attr="Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community")

    for trace in traces:
        mapPoints = []
        for pt in trace["path"]:
            mapPoints.append([pt["GPSlat"], pt["GPSlng"]])
        folium.PolyLine(mapPoints, smooth_factor=0.0).add_to(m)

    mapPoints = []
    for pt in traces[0]["path"]:
        mapPoints.append([pt["GPSlat"], pt["GPSlng"]])
    folium.PolyLine(mapPoints, color="red", smooth_factor=0.0).add_to(m)

    maps.append(m)

for map in maps:
    display(map)